In [1]:
import numpy as np
import pandas as pd
import re
from itertools import islice
import datetime

from sklearn.feature_extraction import text 
from sklearn.metrics.pairwise import cosine_similarity

from nltk.corpus import stopwords
import nltk

import tensorflow as tf

In [2]:
tf.__version__

'1.3.0'

In [4]:
### Explore ted talk data

In [6]:
## ted_main dataset 

ted_main_df = pd.read_csv('./ted-talks/ted_main.csv')
ted_main_df.shape

(2550, 17)

In [30]:
ted_main_df.tail()

,comments,description,duration,event,film_date,languages,main_speaker,name,num_speaker,published_date,ratings,related_talks,speaker_occupation,tags,title,url,views
2545,17,"Between 2008 and 2016, the United States depor...",476,TED2017,1496707200,4,Duarte Geraldino,Duarte Geraldino: What we're missing in the de...,1,1505851216,"[{'id': 3, 'name': 'Courageous', 'count': 24},...","[{'id': 2596, 'hero': 'https://pe.tedcdn.com/i...",Journalist,"['TED Residency', 'United States', 'community'...",What we're missing in the debate about immigra...,https://www.ted.com/talks/duarte_geraldino_wha...,450430
2546,6,How can you study Mars without a spaceship? He...,290,TED2017,1492992000,3,Armando Azua-Bustos,Armando Azua-Bustos: The most Martian place on...,1,1505919737,"[{'id': 22, 'name': 'Fascinating', 'count': 32...","[{'id': 2491, 'hero': 'https://pe.tedcdn.com/i...",Astrobiologist,"['Mars', 'South America', 'TED Fellows', 'astr...",The most Martian place on Earth,https://www.ted.com/talks/armando_azua_bustos_...,417470
2547,10,Science fiction visions of the future show us ...,651,TED2017,1492992000,1,Radhika Nagpal,Radhika Nagpal: What intelligent machines can ...,1,1506006095,"[{'id': 1, 'name': 'Beautiful', 'count': 14}, ...","[{'id': 2346, 'hero': 'https://pe.tedcdn.com/i...",Robotics engineer,"['AI', 'ants', 'fish', 'future', 'innovation',...",What intelligent machines can learn from a sch...,https://www.ted.com/talks/radhika_nagpal_what_...,375647
2548,32,In an unmissable talk about race and politics ...,1100,TEDxMileHigh,1499472000,1,Theo E.J. Wilson,Theo E.J. Wilson: A black man goes undercover ...,1,1506024042,"[{'id': 11, 'name': 'Longwinded', 'count': 3},...","[{'id': 2512, 'hero': 'https://pe.tedcdn.com/i...",Public intellectual,"['Internet', 'TEDx', 'United States', 'communi...",A black man goes undercover in the alt-right,https://www.ted.com/talks/theo_e_j_wilson_a_bl...,419309
2549,8,With more than half of the world population li...,519,TED2017,1492992000,1,Karoliina Korppoo,Karoliina Korppoo: How a video game might help...,1,1506092422,"[{'id': 21, 'name': 'Unconvincing', 'count': 2...","[{'id': 2682, 'hero': 'https://pe.tedcdn.com/i...",Game designer,"['cities', 'design', 'future', 'infrastructure...",How a video game might help us build better ci...,https://www.ted.com/talks/karoliina_korppoo_ho...,391721


In [8]:
ted_main_df.columns

Index(['comments', 'description', 'duration', 'event', 'film_date',
       'languages', 'main_speaker', 'name', 'num_speaker', 'published_date',
       'ratings', 'related_talks', 'speaker_occupation', 'tags', 'title',
       'url', 'views'],
      dtype='object')

In [9]:
## Load transcript data

In [11]:
transcript_df = pd.read_csv('./ted-talks/transcripts.csv')
transcript_df.shape

(2467, 2)

In [12]:
transcript_df.head()

,transcript,url
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...


In [15]:
len(transcript_df.transcript[0])
transcript_df.transcript[0]

'Good morning. How are you?(Laughter)It\'s been great, hasn\'t it? I\'ve been blown away by the whole thing. In fact, I\'m leaving.(Laughter)There have been three themes running through the conference which are relevant to what I want to talk about. One is the extraordinary evidence of human creativity in all of the presentations that we\'ve had and in all of the people here. Just the variety of it and the range of it. The second is that it\'s put us in a place where we have no idea what\'s going to happen, in terms of the future. No idea how this may play out.I have an interest in education. Actually, what I find is everybody has an interest in education. Don\'t you? I find this very interesting. If you\'re at a dinner party, and you say you work in education — Actually, you\'re not often at dinner parties, frankly.(Laughter)If you work in education, you\'re not asked.(Laughter)And you\'re never asked back, curiously. That\'s strange to me. But if you are, and you say to somebody, you

In [17]:
len(list(set(ted_main_df.url)-set(transcript_df.url)))

86

In [22]:
## Join two data sets on url column

ted_transcript_df = ted_main_df[['url','title']].join(transcript_df,on='url',how='right', lsuffix='_left', rsuffix='_right')
ted_transcript_df.shape

(2467, 5)

In [36]:
columns = ['url','title','film_date','published_date','speaker_occupation','tags','name','ratings','description','views','comments']
ted_talk_transcript_df = transcript_df.merge(ted_main_df[columns],on='url',how='left')
ted_talk_transcript_df.head()

,transcript,url,title,film_date,published_date,speaker_occupation,tags,name,ratings,description,views,comments
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...,Do schools kill creativity?,1140825600,1151367060,Author/educator,"['children', 'creativity', 'culture', 'dance',...",Ken Robinson: Do schools kill creativity?,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...",Sir Ken Robinson makes an entertaining and pro...,47227110,4553
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...,Averting the climate crisis,1140825600,1151367060,Climate advocate,"['alternative energy', 'cars', 'climate change...",Al Gore: Averting the climate crisis,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...",With the same humor and humanity he exuded in ...,3200520,265
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...,Simplicity sells,1140739200,1151367060,Technology columnist,"['computers', 'entertainment', 'interface desi...",David Pogue: Simplicity sells,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...",New York Times columnist David Pogue takes aim...,1636292,124
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...,Greening the ghetto,1140912000,1151367060,Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Majora Carter: Greening the ghetto,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","In an emotionally charged talk, MacArthur-winn...",1697550,200
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...,The best stats you've ever seen,1140566400,1151440680,Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",Hans Rosling: The best stats you've ever seen,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...",You've never seen data presented like this. Wi...,12005869,593


In [38]:
ted_talk_transcript_df.shape

(2467, 12)

In [39]:
## Convert Unix timestamp to Human readable format


ted_talk_transcript_df['film_date'] = ted_talk_transcript_df['film_date'].apply(lambda x: datetime.datetime.fromtimestamp( int(x)).strftime('%d-%m-%Y'))
ted_talk_transcript_df['published_date'] = ted_talk_transcript_df['published_date'].apply(lambda x: datetime.datetime.fromtimestamp( int(x)).strftime('%d-%m-%Y'))

ted_talk_transcript_df.head()

,transcript,url,title,film_date,published_date,speaker_occupation,tags,name,ratings,description,views,comments
0,Good morning. How are you?(Laughter)It's been ...,https://www.ted.com/talks/ken_robinson_says_sc...,Do schools kill creativity?,25-02-2006,27-06-2006,Author/educator,"['children', 'creativity', 'culture', 'dance',...",Ken Robinson: Do schools kill creativity?,"[{'id': 7, 'name': 'Funny', 'count': 19645}, {...",Sir Ken Robinson makes an entertaining and pro...,47227110,4553
1,"Thank you so much, Chris. And it's truly a gre...",https://www.ted.com/talks/al_gore_on_averting_...,Averting the climate crisis,25-02-2006,27-06-2006,Climate advocate,"['alternative energy', 'cars', 'climate change...",Al Gore: Averting the climate crisis,"[{'id': 7, 'name': 'Funny', 'count': 544}, {'i...",With the same humor and humanity he exuded in ...,3200520,265
2,"(Music: ""The Sound of Silence,"" Simon & Garfun...",https://www.ted.com/talks/david_pogue_says_sim...,Simplicity sells,24-02-2006,27-06-2006,Technology columnist,"['computers', 'entertainment', 'interface desi...",David Pogue: Simplicity sells,"[{'id': 7, 'name': 'Funny', 'count': 964}, {'i...",New York Times columnist David Pogue takes aim...,1636292,124
3,If you're here today — and I'm very happy that...,https://www.ted.com/talks/majora_carter_s_tale...,Greening the ghetto,26-02-2006,27-06-2006,Activist for environmental justice,"['MacArthur grant', 'activism', 'business', 'c...",Majora Carter: Greening the ghetto,"[{'id': 3, 'name': 'Courageous', 'count': 760}...","In an emotionally charged talk, MacArthur-winn...",1697550,200
4,"About 10 years ago, I took on the task to teac...",https://www.ted.com/talks/hans_rosling_shows_t...,The best stats you've ever seen,22-02-2006,28-06-2006,Global health expert; data visionary,"['Africa', 'Asia', 'Google', 'demo', 'economic...",Hans Rosling: The best stats you've ever seen,"[{'id': 9, 'name': 'Ingenious', 'count': 3202}...",You've never seen data presented like this. Wi...,12005869,593


In [42]:
ted_talk_transcript_df['dis_quo'] = ted_talk_transcript_df['comments']/ted_talk_transcript_df['views']

In [43]:
ted_talk_transcript_df.to_csv('ted_talk_transcript.csv',index=False)

In [3]:
ted_talk_transcript_df = pd.read_csv('ted_talk_transcript.csv')
ted_talk_transcript_df.shape

(2467, 13)

In [6]:
# %matplotlib inline

# import matplotlib.pyplot as plt
# import seaborn as sns

In [8]:
## Here using unigram to find the TF-IDF matrix
Text=ted_talk_transcript_df['transcript'].tolist()
tfidf=text.TfidfVectorizer(input=Text,stop_words="english")
matrix=tfidf.fit_transform(Text)

In [9]:
matrix.shape

(2467, 58489)

In [10]:
### Get Similarity Scores using cosine similarity
sim_unigram=cosine_similarity(matrix)

In [11]:
sim_unigram.shape

(2467, 2467)

In [12]:
sim_unigram[0]

array([ 1.        ,  0.15310631,  0.17626477, ...,  0.07427816,
        0.1408498 ,  0.06360819])

In [ ]:
### Measure the similarities among the talks

In [14]:
def get_similar_articles(x):
    return ",".join(ted_talk_transcript_df['title'].loc[x.argsort()[-5:-1]])

ted_talk_transcript_df['similar_articles_unigram']=[get_similar_articles(x) for x in sim_unigram]

In [20]:
ted_talk_transcript_df['title'].str.replace("_"," ").str.upper().str.strip()[100]

'THE GENESIS OF GOOGLE'

In [21]:
ted_talk_transcript_df['similar_articles_unigram'].str.replace("_"," ").str.upper().str.split("\n")[100]

['WHAT DOES THE FUTURE HOLD? 11 CHARACTERS OFFER QUIRKY ANSWERS,PLAY! EXPERIMENT! DISCOVER!,HOW TO RUN A COMPANY WITH (ALMOST) NO RULES,THE WEB AS A CITY']

### Apply word2vec to measure the similarities

In [5]:

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ashis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
ted_talk_transcript_df.transcript[:2]

0    Good morning. How are you?(Laughter)It's been ...
1    Thank you so much, Chris. And it's truly a gre...
Name: transcript, dtype: object

In [7]:
## Remove unwanted text and stop words
## A list of contractions from http://stackoverflow.com/questions/19790188/expanding-english-language-contractions-in-python

contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"It's":"It is",
"I've":"I have",
"I'm":"I am",
"That's":"That is"
    
}

In [8]:
stops = set(stopwords.words("english"))

In [9]:
def clean_transcript(text):
    
    ## Replace contractions
    
    text = text.split()
    new_text = []
    for word in text:
        
        if word in contractions:
            new_text.append(contractions[word])
        else:
            new_text.append(word)
            
    text = " ".join(new_text)
    
    text = re.sub(r'[—_"\-;%()|+&=*%.,!?:#$@\[\]/]', ' ', text)
    text = text.split()
    text = [w for w in text if not w in stops]
    text = " ".join(text)
    
    return text
    

In [10]:
clean_transcript(ted_talk_transcript_df.transcript[0])

"Good morning How Laughter It's great I blown away whole thing In fact I leaving Laughter There three themes running conference relevant I want talk One extraordinary evidence human creativity presentations people Just variety range The second put us place idea going happen terms future No idea may play I interest education Actually I find everybody interest education Don't I find interesting If dinner party say work education Actually often dinner parties frankly Laughter If work education asked Laughter And never asked back curiously That strange But say somebody know say What say work education see blood run face They're like Oh God know Why Laughter My one night week Laughter But ask education pin wall Because one things goes deep people I right Like religion money things So I big interest education I think We huge vested interest partly education meant take us future cannot grasp If think children starting school year retiring 2065 Nobody clue despite expertise parade past four da

In [11]:
ted_talk_transcript_df['clean_transcript'] = ted_talk_transcript_df['transcript'].apply(lambda x : clean_transcript(x))


In [13]:
ted_talk_transcript_df.shape

(2467, 14)

In [14]:
ted_talk_transcript_df.to_csv('ted_talk_transcript_cleaned.csv',index = False)

In [15]:
# Clean the summaries and texts

clean_summaries = ted_talk_transcript_df.clean_transcript
len(clean_summaries)

2467

In [20]:
type(list(clean_summaries))
clean_summaries = list(clean_summaries)
clean_summaries[0]

"Good morning How Laughter It's great I blown away whole thing In fact I leaving Laughter There three themes running conference relevant I want talk One extraordinary evidence human creativity presentations people Just variety range The second put us place idea going happen terms future No idea may play I interest education Actually I find everybody interest education Don't I find interesting If dinner party say work education Actually often dinner parties frankly Laughter If work education asked Laughter And never asked back curiously That strange But say somebody know say What say work education see blood run face They're like Oh God know Why Laughter My one night week Laughter But ask education pin wall Because one things goes deep people I right Like religion money things So I big interest education I think We huge vested interest partly education meant take us future cannot grasp If think children starting school year retiring 2065 Nobody clue despite expertise parade past four da

In [18]:
def count_words(count_dict, text):
    
    '''Count the number of occurrences of each word in a set of text'''
    
    for sentence in text:
        for word in sentence.split():
            if word not in count_dict:
                count_dict[word] = 1
            else:
                count_dict[word] += 1

In [21]:
# Find the number of times each word was used and the size of the vocabulary

# Find the number of times each word was used and the size of the vocabulary
word_counts = {}

count_words(word_counts, clean_summaries)

In [23]:
len(word_counts)

71910

In [24]:
# Load Conceptnet Numberbatch's (CN) embeddings, similar to GloVe, but probably better 
# (https://github.com/commonsense/conceptnet-numberbatch)
embeddings_index = {}
with open('/home/ashis/ashis/Deep_Learning/Text_Summarization/Text-Summarization-with-Amazon-Reviews/numberbatch-en-17.02.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings:', len(embeddings_index))

Word embeddings: 484557


In [27]:
# Find the number of words that are missing from CN, and are used more than our threshold.
missing_words = 0
threshold = 10

for word, count in word_counts.items():
    if count > threshold:
        if word not in embeddings_index:
            missing_words += 1
            
missing_ratio = round(missing_words/len(word_counts),4)*100
            
print("Number of words missing from CN:", missing_words)
print("Percent of words that are missing from vocabulary: {}%".format(missing_ratio))

Number of words missing from CN: 3394
Percent of words that are missing from vocabulary: 4.72%


In [28]:
# Limit the vocab that we will use to words that appear ≥ threshold or are in CN

#dictionary to convert words to integers
vocab_to_int = {} 

value = 0
for word, count in word_counts.items():
    if count >= threshold or word in embeddings_index:
        vocab_to_int[word] = value
        value += 1

# Special tokens that will be added to our vocab
codes = ["<UNK>","<PAD>","<EOS>","<GO>"]   

# Add codes to vocab
for code in codes:
    vocab_to_int[code] = len(vocab_to_int)

    
# Dictionary to convert integers to words
int_to_vocab = {}
for word, value in vocab_to_int.items():
    int_to_vocab[value] = word

usage_ratio = round(len(vocab_to_int) / len(word_counts),4)*100

print("Total number of unique words:", len(word_counts))
print("Number of words we will use:", len(vocab_to_int))
print("Percent of words we will use: {}%".format(usage_ratio))

Total number of unique words: 71910
Number of words we will use: 43828
Percent of words we will use: 60.95%


In [29]:

def take(n, iterable):
    "Return first n items of the iterable as a list"
    return list(islice(iterable, n))

In [30]:
n_items = take(10, vocab_to_int.items())
n_items

[('horns', 0),
 ('revolutions', 1),
 ('perforate', 2),
 ('anguilliform', 7016),
 ('megatrend', 4),
 ('rue', 5),
 ('flexion', 6),
 ('wrongful', 7),
 ('series', 8),
 ('vines', 9)]

In [31]:
int_n_items = take(10, int_to_vocab.items())
int_n_items

[(0, 'horns'),
 (1, 'revolutions'),
 (2, 'perforate'),
 (3, 'bigfin'),
 (4, 'megatrend'),
 (5, 'rue'),
 (6, 'flexion'),
 (7, 'wrongful'),
 (8, 'series'),
 (9, 'vines')]

In [32]:
# Need to use 300 for embedding dimensions to match CN's vectors.
embedding_dim = 300
nb_words = len(vocab_to_int)

# Create matrix with default values of zero
word_embedding_matrix = np.zeros((nb_words, embedding_dim), dtype=np.float32)
print("word_embedding_matrix shape :",word_embedding_matrix.shape)
for word, i in vocab_to_int.items():
    if word in embeddings_index:
        word_embedding_matrix[i] = embeddings_index[word]
    else:
        # If word not in CN, create a random embedding for it
        new_embedding = np.array(np.random.uniform(-1.0, 1.0, embedding_dim))
        embeddings_index[word] = new_embedding
        word_embedding_matrix[i] = new_embedding

# Check if value matches len(vocab_to_int)
print(len(word_embedding_matrix))

word_embedding_matrix shape : (43828, 300)
43828


In [33]:
def convert_to_ints(text, word_count, unk_count, eos=False):
    
    '''Convert words in text to an integer.
       If word is not in vocab_to_int, use UNK's integer.
       Total the number of words and UNKs.
       Add EOS token to the end of texts
    '''
    
    ints = []
    for sentence in text:
        sentence_ints = []
        for word in sentence.split():
            word_count += 1
            if word in vocab_to_int:
                sentence_ints.append(vocab_to_int[word])
            else:
                sentence_ints.append(vocab_to_int["<UNK>"])
                unk_count += 1
        if eos:
            sentence_ints.append(vocab_to_int["<EOS>"])
        ints.append(sentence_ints)
        
    return ints, word_count, unk_count

In [34]:
# Apply convert_to_ints to clean_summaries and clean_texts
word_count = 0
unk_count = 0

int_texts, word_count, unk_count = convert_to_ints(clean_summaries, word_count, unk_count, eos=True)
unk_percent = round(unk_count/word_count,4)*100


In [35]:
print("Total number of words in headlines:", word_count)
print("Total number of UNKs in headlines:", unk_count)
print("Percent of words that are UNK: {}%".format(unk_percent))

Total number of words in headlines: 2765727
Total number of UNKs in headlines: 58860
Percent of words that are UNK: 2.13%


In [36]:
def create_lengths(text):
    '''Create a data frame of the sentence lengths from a text'''
    lengths = []
    for sentence in text:
        lengths.append(len(sentence))
    return pd.DataFrame(lengths, columns=['counts'])

In [37]:
lengths_texts = create_lengths(int_texts)

print("Texts:")
print(lengths_texts.describe())

Texts:
            counts
count  2467.000000
mean   1122.089177
std     510.402839
min       3.000000
25%     751.000000
50%    1122.000000
75%    1477.000000
max    4859.000000


In [38]:
# Inspect the length of texts
print(np.percentile(lengths_texts.counts, 90))
print(np.percentile(lengths_texts.counts, 95))
print(np.percentile(lengths_texts.counts, 99))


1734.0
1906.0
2367.06


In [42]:
print(np.percentile(lengths_texts.counts, 99))

2367.06


In [ ]:
"""

Step-1: Consider all the talks till length of 2500
Step-2: If length of the talk is less than 2500, append <UNK> to make it of length 2500
Step-3: Then create an aggregation of all the embeddings to get a final one
Step-4: Create a matrix of dimension 2467 * 300 , i.e 300 embeddings dimension
Step-5: Find the top five similar talks by taking the cosine similarities of the aggregated embeddings

"""

In [85]:
def create_embeddings(ted_talk_words):
    
    temp_embeddings_arr =[]
    
    for word in ted_talk_words:
        
        try:
            temp_embeddings_arr.append(embeddings_index[word])
            
        except KeyError:
            temp_embeddings_arr.append(embeddings_index['<UNK>'])
            
        #print(np.array(temp_embeddings_arr).shape)
        return np.mean(np.array(temp_embeddings_arr),axis=0)
   

In [86]:
## Create Aggegated Embeddings for similarity(cosine) measure

def create_aggregated_embeddings(ted_talks):
    
    max_sentence_length = 2500
    ted_talk_final_embeddings = []
    
    for ted_talk in ted_talks:
        
        ted_talk_words = ted_talk.split()
        
        if len(ted_talk_words)>= 2500:
            
            ## Get only first 2500 words from the list
            ted_talk_words = ted_talk_words[:2500]
            
            ## Call to create embedding function
            ted_talk_final_embeddings.append(create_embeddings(ted_talk_words))
            
        else:
            
            ## Append <UNK> to the list to make it 2500
            ted_talk_words = [ted_talk_words + ['<UNK>'] * (max_sentence_length - len(ted_talk_words))][0]
            
            ## Call to create embedding function
            ted_talk_final_embeddings.append(create_embeddings(ted_talk_words))

    return np.array(ted_talk_final_embeddings)
        

In [90]:
final_embeddings = create_aggregated_embeddings(clean_summaries)

In [91]:
final_embeddings.shape

(2467, 300)

In [92]:

sim_unigram=cosine_similarity(final_embeddings)
sim_unigram.shape

(2467, 2467)

In [97]:
def get_similar_articles(x):
    return ",".join(ted_talk_transcript_df['title'].loc[x.argsort()[-5:-1]])


In [98]:
ted_talk_transcript_df['similar_articles_unigram']=[get_similar_articles(x) for x in sim_unigram]

In [103]:
ted_talk_transcript_df['similar_articles_unigram'].str.replace("_"," ").str.upper().str.split("\n")[39]

["WE NEED TO TALK ABOUT AN INJUSTICE,THE ACCELERATING POWER OF TECHNOLOGY,HOW MARS MIGHT HOLD THE SECRET TO THE ORIGIN OF LIFE,WHO WOULD THE REST OF THE WORLD VOTE FOR IN YOUR COUNTRY'S ELECTION?"]